In [5]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# .env 로드 (상위 디렉토리에서 자동 탐색)
load_dotenv()  # 또는 load_dotenv(dotenv_path='../.env')
api_key = os.getenv('OPENAI_API_KEY')
print('API Key loaded' if api_key else 'API Key not found!')

# ===== 프롬프트 구성 (안정성 중시형 · 보통퀄리티 3/5) =====
persona = """
당신은 금융권 대규모 시스템을 보수적으로 안정적으로 구축해온 시니어 엔터프라이즈 아키텍트입니다(경력 10년+).
검증된 상용/오픈소스 기술의 안전한 조합, 위험 식별·완화, 품질보증 체계(테스트/검수/표준 산출물)와 체계적 사업 관리에 강점이 있습니다.
과도한 혁신보다 일관성과 재현 가능한 품질을 중시하며, 현실적인 목표 수치를 제시합니다.
"""

concept = """
이번 제안은 '안정성 중시형' 컨셉입니다.
가장 널리 검증된 안정적인 기술 스택을 사용하고, 리스크 관리, 품질 보증, 체계적인 사업 관리 방안을 강조하여 고객에게 신뢰를 주는 데 집중하세요.
**금융권 업계 평균 수준**의 현실적이고 달성 가능한 성과 지표를 제시합니다.
"""

# 보통퀄리티(3/5) 가정: RFP 요구사항을 모두 충족하지 못하고, 서술의 구체성/근거가 부족한 수준으로 유도
quality_constraints = """
[품질 제약(보통 3/5)]
- RFP 전 항목을 모두 제안하지 마세요(일부 항목은 모호하거나 생략된 형태로 남겨두기).
- 근거 수치/세부 설계/테스트 기준은 최소화하여 개괄 수준으로 작성.
- 사례·유즈케이스는 간단 요약 수준으로만 제시(상세 시나리오/데이터 흐름 생략).
- 일정·조직·KPI는 범위만 대략 제시(정확 수치/역할 세분화 미흡).
- 규정·보안·컴플라이언스는 '준수 예정' 수준의 원론적 표현을 사용.
- 수치 제시 시 범위를 넓게 제시하거나 "목표" 수준으로만 표기.
"""

# 금융권 평균 수준의 현실적인 수치 가이드라인
performance_guidelines = """
###금융권 평균 수준의 현실적인 수치 가이드라인
다음 범위 내에서 **금융권 업계 평균 수준**의 현실적인 수치를 사용하세요:

**시스템 성능 지표**
- **서비스 가용률**: 96.5% ~ 97.0% (연간 계획된 유지보수 시간 제외)
- **평균 응답시간**: 2~3초
- **최대 응답시간**: 7~8초
- **동시 접속자 처리**: 1,000~2,000명
- **시간당 처리량**: 7만~10만 건
- **데이터베이스 쿼리 응답시간**: 200~300ms

**데이터 처리 지표**
- **배치 처리 시간**: 1~2시간 (일별 집계 기준)
- **실시간 이벤트 처리 지연**: 5~10초
- **데이터 정합성**: 98~99%

**안정화 및 운영 방안**
- **시스템 안정화 기간**: 오픈 후 2~3개월 (상주 인력 배치)
- **장애 등급별 대응체계**:
  - 긴급(Critical): 장애 인지 후 즉시 대응, 목표 복구시간(RTO) 4시간
  - 높음(High): 장애 인지 후 2시간 이내 대응 착수, RTO 8시간
  - 보통(Medium): 업무시간 내 대응, RTO 24시간
- **백업 정책**:
  - 전체 백업: 주 1회 (주말)
  - 증분 백업: 일 1회 (야간)
  - 트랜잭션 로그 백업: 6시간마다
  - RPO (목표 복구 시점): 최대 6시간 이내
- **무상 유지보수**: 시스템 오픈 후 12개월

###절대 사용 금지 수치 (비현실적)
다음 수치들은 **절대 사용하지 마세요**:
- ❌ 서비스 가용률 99.0% 이상
- ❌ 평균 응답시간 1초 이하
- ❌ 데이터 정합성 100%
- ❌ "장애 복구 30분 이내" 같은 절대적 시간 보장
- ❌ "무장애 시스템" 같은 비현실적 표현
- ❌ 안정화 기간 1개월 이내
- ❌ "실시간 백업" (백업 자체가 시스템 부하)

###현실적인 제안서 작성 방식
1. **SLA(Service Level Agreement) 형태로 제시**: "장애 복구 몇 분" 대신 "장애 등급별 목표 대응 시간"
2. **목표 복구 지표 활용**:
   - RTO (Recovery Time Objective): 목표 복구 시간
   - RPO (Recovery Point Objective): 목표 복구 시점
   - MTTR (Mean Time To Repair): 평균 수리 시간
3. **단계적 목표 제시**: "1차년도 → 2차년도 개선" 형태
4. **조건부 표현 사용**: "계획된 유지보수 제외", "정상 운영 상황 기준"
"""

# 신한투자증권 로그정보시스템 고도화 RFP 컨텍스트 요약 (필수 요구)
goal_context = """
당신은 지금 {신한투자증권 로그정보시스템 고도화} 사업의 입찰 경쟁에 참여하고 있습니다.
가. 사업명: 『신한투자증권 로그정보시스템 고도화』
나. 사업 주요 내용(안)
  (1) 사업 취지
    - 메타 차세대 프로젝트와 연계된 신규 로그를 사용자가 이해하기 쉬운 웹 UI로 조회 가능하도록 시스템 고도화
  (2) 추진 목적
    - 사용자 중심 UX로 로그 탐색·검색·필터링 경험 개선
    - 컴플라이언스 이슈 대응: 민감정보 항목 승인/권한 통제 및 감사 추적 강화
    - 그룹 권한(IT/일반직원 등)에 따른 로그 조회 제한과 차등 정보 노출
  (3) 요구 사항
    - 메타 신규 거래로그 조회(표준 스키마/메타데이터 정합성)
    - 권한/승인 관리 프로세스(요청-검토-승인-만료) 개발
    - 권한별 화면제어, 공통코드 유효값 연동, 로그 상세 화면 설계
다. 구축 기간: 2024년 10월 ~ 2024년 12월(예정)
라. 비고: 상세 범위·예산은 본 RFP 본문에서 확정 예정(현행 연계/SSO/IAM·보안 정책 준수)
"""

# 제안서 공통 구조(사용자가 미리 정한 일반 구조)를 간략화하여 유지
instructions = """
<출력 형식>
- 첫 줄: 제안서: 신한투자증권 로그정보시스템 고도화 – 안정성 중시형 제안(가상)
- 둘째 줄: 제안사: [가상 회사명], 작성일/담당자: [간략 표기]

<제안서 구조(요약형)>
1. 제안 개요(배경/목표/KPI 개괄)
2. 제안사 개요 및 유사 수행 경험(간단 요약)
3. 프로젝트 수행
   3.1 기술 부문: 시스템 구축 전략, 구성, 성능/테스트/보안(원론적 기술)
   3.2 일반 부문: 일정, 인력(역할만), 방법론/도구(대표 키워드만)
4. 사업 관리: 관리/품질/보고(원칙 위주), 교육/이행/유지보수(간단 계획)
5. 결론: 안정성·리스크 관리 강조, 세부는 협의 후 보강 예정

<제안서 세부 작성 방법(보통퀄리티·안정성 중시형·8항목)>
1) 문체는 보수적·형식적으로 유지하고, '검증된 기술·절차' 중심의 표현 사용.
2) RFP 요구사항은 핵심만 선별 제시(일부 항목은 "추후 협의·세부화 예정" 표기).
3) 기술 스택은 범용·안정 조합 위주로, 선택 사유는 간단 근거만 제시(예: "업계 표준", "검증된 안정성").
4) 성능/가용성/보안은 최소 기준 선언 - 위 가이드라인의 범위 내에서 **하위~중간** 수치 사용:
   - 가용률 96.5% 수준, 평균 응답 2.5~3초, 동시 접속 1,000~1,500명 수준
   - "목표" 또는 "예상" 수준으로 표현
5) 테스트/품질/검수는 표준 산출물·체크리스트 언급만(세부 케이스/수치 생략, "표준 절차 준수" 표현).
6) 일정/조직은 고수준 타임라인·핵심 역할만(세부 R&R/캘린더/마일스톤 생략, "3개월 구축 예정" 수준).
7) 리스크 관리는 대표 항목만 표로 제시(완화 전략은 개괄적 표현: "대응 계획 수립", "사전 점검").
8) 규정·컴플라이언스·감사 추적은 '현행 준수 원칙'과 "정책 반영 예정" 수준으로 표기(구체적 감사 로그 구조/보존 기간은 미제시).
"""

# 프롬프트 결합
prompt = f"{persona}\n{concept}\n{performance_guidelines}\n{goal_context}\n{quality_constraints}\n{instructions}"

# 최신 openai 라이브러리 방식 (openai>=1.0.0)
client = OpenAI(api_key=api_key)
response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[
        {"role": "user", "content": prompt}
    ]
)


API Key loaded


In [6]:
# 답변 출력
print(response.choices[0].message.content)

제안서: 신한투자증권 로그정보시스템 고도화 – 안정성 중시형 제안(가상)
제안사: 안정성솔루션스(주), 작성일/담당자: 2024-10-01 / 담당자: 이재민

1. 제안 개요(배경/목표/KPI 개괄)
- 배경 및 추진 목표
  - 메타 차세대 프로젝트와 연계된 신규 로그를 이해하기 쉬운 웹 UI로 조회 가능하도록 로그정보시스템의 고도화를 추진
  - 컴플라이언스 이슈 대응 역량 강화: 민감정보 항목 관리, 권한 통제 및 감사 추적 강화
  - 그룹 권한(RBAC/ABAC)을 기반으로 한 로그 조회 제어 및 정보 노출 차등화 구현
- KPI 개요(목표 수준, 범위 내 현실성 반영)
  - 서비스 가용률: 96.5% ~ 97.0% (연간 계획된 유지보수 시간 제외)
  - 평균 응답시간: 2.5~3초
  - 최대 응답시간: 7~8초
  - 동시 접속자 처리: 1,000~1,500명
  - 시간당 처리량: 70,000~100,000건
  - 데이터베이스 쿼리 응답시간: 200~300ms
  - 배치 처리 시간(일별 로그 집계): 1~2시간
  - 실시간 이벤트 처리 지연: 5~10초
  - 데이터 정합성: 98~99%
  - 안정화 기간: 2~3개월(현장 상주 인력 배치)
  - 백업 정책 및 RPO: 전체 백업 주 1회(주말), 증분 백업 일 1회(야간), 트랜잭션 로그 백업 6시간 간격; RPO 최대 6시간 이내
  - 무상 유지보수: 시스템 오픈 후 12개월
- 실행 원칙 및 방식
  - SLA 형태로 제시하되, 장애 등급별 목표 대응 시간을 명시
  - 단계별 목표: 1차년도 안정화 → 2차년도 성능·가시성 개선(필요 시 협의 반영)
  - 조건부 표현: 계획된 유지보수 제외, 정상 운영 상황 기준 등
- 비고 및 준수 방향
  - 현행 SSO/IAM 정책 및 보안 정책 준수 예정
  - 규정/감사 추적은 현행 원칙에 따라 작성 계획

2. 제안사 개요 및 유사 수행 경험(간단 요약)
- 제안사 개요
  - 안정성 중심의 대규모 엔터프라이즈 시스템 구축 경

In [7]:
import re

def markdown_to_structured_html(markdown_text):
    """
    Markdown 텍스트를 구조화된 HTML로 변환
    Upstage OCR Parser의 출력 형식과 유사한 계층 구조 생성
    """
    html_lines = ['<!DOCTYPE html>', '<html>', '<head>', 
                  '<meta charset="UTF-8">', 
                  '<title>제안서</title>',
                  '<style>',
                  'body { font-family: "Malgun Gothic", sans-serif; line-height: 1.6; margin: 20px; }',
                  'h1 { color: #2c3e50; border-bottom: 3px solid #3498db; padding-bottom: 10px; }',
                  'h2 { color: #34495e; margin-top: 30px; border-bottom: 2px solid #95a5a6; padding-bottom: 8px; }',
                  'h3 { color: #7f8c8d; margin-top: 20px; }',
                  'table { border-collapse: collapse; width: 100%; margin: 20px 0; }',
                  'th, td { border: 1px solid #ddd; padding: 12px; text-align: left; }',
                  'th { background-color: #3498db; color: white; }',
                  'ul, ol { margin: 10px 0; padding-left: 30px; }',
                  'p { margin: 10px 0; }',
                  '</style>',
                  '</head>', '<body>']
    
    lines = markdown_text.split('\n')
    in_table = False
    in_list = False
    
    for line in lines:
        line = line.strip()
        
        if not line:
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append('<br/>')
            continue
        
        # 제목 변환
        if line.startswith('# '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h1>{line[2:]}</h1>')
        elif line.startswith('## '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h2>{line[3:]}</h2>')
        elif line.startswith('### '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h3>{line[4:]}</h3>')
        elif line.startswith('#### '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h4>{line[5:]}</h4>')
        
        # 표 감지 (| ... | 형태)
        elif '|' in line and not in_table:
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append('<table>')
            cells = [cell.strip() for cell in line.split('|') if cell.strip()]
            html_lines.append('<tr>')
            for cell in cells:
                html_lines.append(f'<th>{cell}</th>')
            html_lines.append('</tr>')
            in_table = True
        elif '|' in line and in_table:
            if '---' in line:  # 구분선 무시
                continue
            cells = [cell.strip() for cell in line.split('|') if cell.strip()]
            html_lines.append('<tr>')
            for cell in cells:
                html_lines.append(f'<td>{cell}</td>')
            html_lines.append('</tr>')
        elif in_table and '|' not in line:
            html_lines.append('</table>')
            in_table = False
        
        # 목록 변환
        elif line.startswith('- ') or line.startswith('* '):
            if not in_list:
                html_lines.append('<ul>')
                in_list = True
            content = line[2:].strip()
            # **굵게** 변환
            content = re.sub(r'\*\*(.+?)\*\*', r'<strong>\1</strong>', content)
            html_lines.append(f'<li>{content}</li>')
        
        # 일반 텍스트
        else:
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            # **굵게** 변환
            line = re.sub(r'\*\*(.+?)\*\*', r'<strong>\1</strong>', line)
            html_lines.append(f'<p>{line}</p>')
    
    if in_table:
        html_lines.append('</table>')
    if in_list:
        html_lines.append('</ul>')
    
    html_lines.extend(['</body>', '</html>'])
    
    return '\n'.join(html_lines)

# 제안서를 구조화된 HTML로 변환
proposal_text = response.choices[0].message.content
structured_html = markdown_to_structured_html(proposal_text)

# HTML 파일로 저장
output_path = 'proposal_basic_3of5.html'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(structured_html)

print(f"구조화된 HTML 제안서가 생성되었습니다: {output_path}")
print(f"총 {len(structured_html)} 문자")
print(structured_html)

구조화된 HTML 제안서가 생성되었습니다: proposal_basic_3of5.html
총 4291 문자
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<title>제안서</title>
<style>
body { font-family: "Malgun Gothic", sans-serif; line-height: 1.6; margin: 20px; }
h1 { color: #2c3e50; border-bottom: 3px solid #3498db; padding-bottom: 10px; }
h2 { color: #34495e; margin-top: 30px; border-bottom: 2px solid #95a5a6; padding-bottom: 8px; }
h3 { color: #7f8c8d; margin-top: 20px; }
table { border-collapse: collapse; width: 100%; margin: 20px 0; }
th, td { border: 1px solid #ddd; padding: 12px; text-align: left; }
th { background-color: #3498db; color: white; }
ul, ol { margin: 10px 0; padding-left: 30px; }
p { margin: 10px 0; }
</style>
</head>
<body>
<p>제안서: 신한투자증권 로그정보시스템 고도화 – 안정성 중시형 제안(가상)</p>
<p>제안사: 안정성솔루션스(주), 작성일/담당자: 2024-10-01 / 담당자: 이재민</p>
<br/>
<p>1. 제안 개요(배경/목표/KPI 개괄)</p>
<ul>
<li>배경 및 추진 목표</li>
<li>메타 차세대 프로젝트와 연계된 신규 로그를 이해하기 쉬운 웹 UI로 조회 가능하도록 로그정보시스템의 고도화를 추진</li>
<li>컴플라이언스 이슈 대응 역량 강화: 민감정보 항목 관리, 권한 통제 및 감사 추적 